In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-26T11:34:44.705241-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn
import gc
import os

In [5]:
%watermark --iversions

sklearn: 1.2.2
numpy  : 1.23.5
pandas : 1.5.3



In [6]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [7]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values

In [10]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(60000, 10)

In [11]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold accuracy: 0.9045833333333333
Fitting fold 2
Fold accuracy: 0.901
Fitting fold 3
Fold accuracy: 0.9031666666666667
Fitting fold 4
Fold accuracy: 0.9
Fitting fold 5
Fold accuracy: 0.9033333333333333
CPU times: user 26min 38s, sys: 13min 43s, total: 40min 22s
Wall time: 1min 21s


In [12]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.9024166666666666


In [13]:
preds = np.argmax(test_preds, axis=1)
preds

array([0, 1, 6, ..., 8, 8, 1])

In [14]:
test.label.values

array([0, 1, 2, ..., 8, 8, 1])

In [15]:
accuracy_score(test.label.values, preds)

0.9078

In [16]:
0.9098

0.9098

In [17]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=9, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9045833333333333
Fitting fold 2
Fold accuracy: 0.9010833333333333
Fitting fold 3
Fold accuracy: 0.9030833333333333
Fitting fold 4
Fold accuracy: 0.9000833333333333
Fitting fold 5
Fold accuracy: 0.90225
0.9022166666666667
0.9074
CPU times: user 23min 4s, sys: 12min 59s, total: 36min 4s
Wall time: 1min 17s


In [18]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=12, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9044166666666666
Fitting fold 2
Fold accuracy: 0.9006666666666666
Fitting fold 3
Fold accuracy: 0.9038333333333334
Fitting fold 4
Fold accuracy: 0.9009166666666667
Fitting fold 5
Fold accuracy: 0.9029166666666667
0.90255
0.909
CPU times: user 23min 45s, sys: 13min 9s, total: 36min 54s
Wall time: 1min 20s


In [19]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=15, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9035
Fitting fold 2
Fold accuracy: 0.8994166666666666
Fitting fold 3
Fold accuracy: 0.9033333333333333
Fitting fold 4
Fold accuracy: 0.9008333333333334
Fitting fold 5
Fold accuracy: 0.903
0.9020166666666667
0.9099
CPU times: user 23min 40s, sys: 12min 53s, total: 36min 34s
Wall time: 1min 17s


In [20]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=16, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.90325
Fitting fold 2
Fold accuracy: 0.8991666666666667
Fitting fold 3
Fold accuracy: 0.90275
Fitting fold 4
Fold accuracy: 0.9010833333333333
Fitting fold 5
Fold accuracy: 0.9029166666666667
0.9018333333333334
0.9097
CPU times: user 24min 4s, sys: 13min 22s, total: 37min 26s
Wall time: 1min 20s


In [21]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=14, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9039166666666667
Fitting fold 2
Fold accuracy: 0.8996666666666666
Fitting fold 3
Fold accuracy: 0.90275
Fitting fold 4
Fold accuracy: 0.9010833333333333
Fitting fold 5
Fold accuracy: 0.9030833333333333
0.9021
0.9096
CPU times: user 24min 3s, sys: 12min 16s, total: 36min 20s
Wall time: 1min 20s


In [22]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=15.1, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9033333333333333
Fitting fold 2
Fold accuracy: 0.8995
Fitting fold 3
Fold accuracy: 0.9034166666666666
Fitting fold 4
Fold accuracy: 0.9009166666666667
Fitting fold 5
Fold accuracy: 0.9030833333333333
0.90205
0.9099
CPU times: user 27min 6s, sys: 11min 31s, total: 38min 38s
Wall time: 1min 22s
